* calc correlation by RBF
* RBFを使用して相関行列を計算する

In [ ]:
!pip install japanize-matplotlib

In [ ]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import random, re

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

import japanize_matplotlib
plt.rcParams['font.family']

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

In [ ]:
img_cnt = 0
def save_img():
    global img_cnt
    img_cnt += 1
    img_name = 'img_{0:03}.png'.format(img_cnt)
    print(img_name)
    plt.savefig(img_name)

# データ 読み込み

In [ ]:
df_out_wgt_user = pd.read_csv('../input/lowcol-sample007-fet2-randinit01/out_user.csv', index_col=0)
X_df_user = df_out_wgt_user
wgt_user = df_out_wgt_user.values
df_out_wgt_user.head()

In [ ]:
df_out_wgt_col = pd.read_csv('../input/lowcol-sample007-fet2-randinit01/out_col.csv', index_col=0)
X_df_col = df_out_wgt_col
wgt_prod = df_out_wgt_col.values
df_out_wgt_col.head()

In [ ]:
df_out_wgt_col.index.values

In [ ]:
X_df = df_out_wgt_user

In [ ]:
'''特徴量算出で計算されたガンマを取得'''
gamma = np.load('../input/lowcol-sample007-fet2-randinit01/out_gamma.npy').tolist()
gamma

In [ ]:
x1 = df_out_wgt_col.values[0,:]
x2 = df_out_wgt_col.values[1,:]
x1, x2

In [ ]:
d2 = ((x1 - x2)**2).sum()
d2

In [ ]:
'''距離類似度算出の関数を定義'''
def calc(target, df, gamma=1, sort=True):
    res = []
    for ee in df.index.values:
        x = df.loc[ee,:]
        d2 = ((target - x)**2).sum()
        res0 = np.exp(-gamma * d2)
        res.append(res0)
    df_out = pd.DataFrame(df.index.values, index=df.index.values, columns=['title'])
    df_out['prob'] = res
    if sort:
        df_out.sort_values('prob', inplace=True, ascending=False)
    return df_out

In [ ]:
res = calc(df_out_wgt_col.values[0,:], df_out_wgt_col, gamma=gamma)
res

In [ ]:
#target_col = df_out_wgt_col.index.values[[True if re.search('^a21001.Q25.', ee) else False for ee in df_out_wgt_col.index.values]]
target_col = df_out_wgt_col.index.values
target_col

In [ ]:
res_list = []
for idx in target_col:
    res = calc(df_out_wgt_col.loc[idx,:].values, df_out_wgt_col, gamma=gamma, sort=False)
    res2 = res
    res_list.append(res2['prob'].values)

In [ ]:
df_res = pd.DataFrame(res_list, index=target_col, columns=target_col)
df_res

In [ ]:
df_res.to_csv('df_res.csv')